In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urban-dictionary-words-dataset/urbandict-word-defs.csv


In [2]:
train = pd.read_csv('/kaggle/input/urban-dictionary-words-dataset/urbandict-word-defs.csv', nrows=100000, error_bad_lines=False)

b'Skipping line 3692: expected 6 fields, saw 7\nSkipping line 5546: expected 6 fields, saw 7\nSkipping line 7198: expected 6 fields, saw 7\nSkipping line 9758: expected 6 fields, saw 7\nSkipping line 13350: expected 6 fields, saw 7\nSkipping line 20000: expected 6 fields, saw 7\nSkipping line 20088: expected 6 fields, saw 7\nSkipping line 21776: expected 6 fields, saw 8\nSkipping line 23826: expected 6 fields, saw 8\nSkipping line 25255: expected 6 fields, saw 7\nSkipping line 25643: expected 6 fields, saw 7\nSkipping line 25777: expected 6 fields, saw 7\nSkipping line 30965: expected 6 fields, saw 7\nSkipping line 35485: expected 6 fields, saw 7\nSkipping line 36022: expected 6 fields, saw 8\nSkipping line 36072: expected 6 fields, saw 7\nSkipping line 40152: expected 6 fields, saw 7\nSkipping line 40695: expected 6 fields, saw 7\nSkipping line 41942: expected 6 fields, saw 7\nSkipping line 43660: expected 6 fields, saw 7\nSkipping line 46529: expected 6 fields, saw 7\nSkipping line 4

In [3]:
#train.columns
new_train = train[['word', 'definition']]

In [4]:
new_train['word'] = new_train.word.str.lower()
new_train['definition'] = new_train.definition.str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# new_train.to_csv('data.csv', index=False, sep='~')

In [5]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import os
from tqdm import tqdm

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [11]:
#new_train.head()
new_train.shape
new_train.columns

Index(['word', 'definition'], dtype='object')

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

In [21]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv

class GlossaryDataset(Dataset):
    def __init__(self, dataframe):
        super().__init__()


        self.data_list = []
        self.end_of_text_token = "<|endoftext|>"
        self.start_of_text_token = "<|startoftext|>"
        
        for i in range(dataframe.shape[0]):
            data_str = f"{self.start_of_text_token} {new_train.iloc[i]['word']} [DEFINE] {new_train.iloc[i]['definition']} {self.end_of_text_token}"
            self.data_list.append(data_str)
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, item):
        return self.data_list[item]

In [22]:
dataset = GlossaryDataset(dataframe=new_train)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [28]:
# max len
max([len(i[0].split()) for i in data_loader])


5

In [29]:
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5
WARMUP_STEPS = 1000
MAX_SEQ_LEN = 500
seq_cnt = 0
batch_count = 0
sum_loss = 0.0

from transformers import AdamW, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)

In [30]:
for epoch in range(EPOCHS):
  print (f'Running {epoch} epoch')

  for idx,joke in enumerate(data_loader):
    joke_ = torch.tensor(tokenizer.encode(joke[0]))
    joke_=joke_.unsqueeze(0).to(device)
    outputs = model(joke_, labels=joke_)
    loss, logits = outputs[:2]
    loss.backward()
    sum_loss += loss.data

    seq_cnt += 1
    if seq_cnt == BATCH_SIZE:
      seq_cnt = 0    
      batch_count += 1
      optimizer.step()
      scheduler.step() 
      optimizer.zero_grad()
      model.zero_grad()

    if batch_count == 100:
      print(f"sum loss {sum_loss}")
      batch_count = 0
      sum_loss = 0.0


Running 0 epoch
sum loss 30025.76953125
sum loss 19658.46875
sum loss 18658.2421875
sum loss 18422.333984375
sum loss 18021.263671875
sum loss 17886.521484375
sum loss 17843.765625
sum loss 17621.337890625
sum loss 17567.53515625
sum loss 17471.611328125
sum loss 17448.0234375
sum loss 17558.81640625
sum loss 17404.85546875
sum loss 17499.3203125
sum loss 17449.3984375
Running 1 epoch
sum loss 17445.13671875
sum loss 17323.49609375
sum loss 17311.849609375
sum loss 17224.064453125
sum loss 17368.869140625
sum loss 17327.068359375
sum loss 17295.970703125
sum loss 17419.18359375


KeyboardInterrupt: 

In [139]:
def choose_from_top_k_top_n(probs, k=2, p=0.8):
    ind = np.argpartition(probs, -k)[-k:]
    top_prob = probs[ind]
    top_prob = {i: top_prob[idx] for idx,i in enumerate(ind)}
    sorted_top_prob = {k: v for k, v in sorted(top_prob.items(), key=lambda item: item[1], reverse=True)}
    
    t=0
    f=[]
    pr = []
    for k,v in sorted_top_prob.items():
      t+=v
      f.append(k)
      pr.append(v)
      if t>=p:
        break
    top_prob = pr / np.sum(pr)
    token_id = np.random.choice(f, 1, p = top_prob)

    return int(token_id)

In [140]:
model.eval()
# model.train()
def predict(sentence):  
  joke_num = 0
  with torch.no_grad():
    for joke_idx in range(1):
        joke_finished = False
        cur_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0).to(device)
        for i in range(500):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]

            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
            if i < 5:
                n = 10
            else:
                n = 5
            next_token_id = choose_from_top_k_top_n(softmax_logits.to('cpu').numpy()) #top-k-top-n sampling
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                joke_finished = True
                break

        if joke_finished:
          
            joke_num = joke_num + 1
            
            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)
            return output_text
        else:
            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)
            return output_text

In [195]:
predict('<|startoftoken|> guh [DEFINE]')

"<|startoftoken|> guh [DEFINE] the word 'guh' means 'good' or 'great'. ;; also, 'guh' is a very common greeting in the south of the united states<|endoftext|>"

In [ ]:
predicted = '<|startoftoken|> rhitard [DEFINE] the most annoying person on earth.<|endoftext|>'
actual = 'idiot' or 'retard'

predicted = '<|startoftoken|> dsl [DEFINE] a dsl is a computer that is used to connect to the internet. ;; a dsl is also used to connect to a network.<|endoftext|>'
actual = 'high speed connection to the internet '

predicted = '<|startoftoken|> janky [DEFINE] a person who is janky<|endoftext|>'
actual = 'undesirable'

predicted = '<|startoftoken|> scrilla [DEFINE] a small, white, round, and round shaped object.<|endoftext|>'
actual = 'cash money'

predicted = '"<|startoftoken|> guh [DEFINE] the word 'guh' means 'good' or 'great'. ;; also, 'guh' is a very common greeting in the south of the united states<|endoftext|>"'
actual = "to be used in replacement of 'uhhh'"

In [165]:
new_train.head(90)

,word,definition
0,janky,undesirable; less-than optimum.
1,slumpin',"low down and funky, but [knee deep] enough to ..."
2,yayeeyay,"affirmation; suggestion of encouragement, appr..."
3,hard-core,anything out of our league that can be good or...
4,brutal,anything that makes you sweat
...,...,...
85,real men,1. guys with chest hair ;; 2. guys who thig th...
86,watchers,1. a group of individuals who monitor the comm...
87,dsl,1. high speed connection to the internet ;; 2....
88,warn,1.an action that is taken on the internet to p...


In [191]:
new_train.iloc[111]['word']

'guh'

In [192]:
new_train.iloc[111]['definition']

"to be used in replacement of 'uhhh'"